In [53]:
import numpy as np

# n=45, s=5, d=1

In [54]:
import numpy as np
from mpmath import mp, mpc

from numpy.polynomial import polynomial as P

def construct_c_mp(n):
    # complex matrix here
    mp.dps = 25
    mp.pretty = True
    mpc.dps = 25
    _shape = (n, n)
    C = mp.matrix(n)

    for p in range(_shape[0]):
        for q in range(_shape[1]):
            if q>=p:
                if p == 0 or q == 0:
                    C[p, q] = 1+0j
                else:
                    C[p, q]=0+mpc(mp.exp(mpc(-2*mp.pi*p*q*1j/n)))
            else:
                C[p, q] = C[q, p]
    return C


def cls_solving_mp(C_1, fake_W):
    # return Q here:
    #_shape = np.transpose(C_1).shape
    _shape = (len(C_1[0, :]), len(C_1))
    Q = mp.ones(_shape[0], _shape[1])
    for i in range(_shape[1]):
        indices = np.where(np.transpose(fake_W)[i]==0)[0]
        _A = mp.zeros(len(indices), _shape[0]-1)
        _b = mp.zeros(len(indices),1)
        for j, index in enumerate(indices):
            _A[j,:] = C_1[index,1:]
            _b[j] = -C_1[index,0]
        _q=mp.lu_solve(_A, _b)
        Q[1:,i] = _q    
        W = C_1 * Q
    return W, fake_W

def search_w_mp(n, s):
    _hat_s = int(2*s+1)

    C = construct_c_mp(n)

    C_1 = C[:, 0:n-_hat_s+1]
    C_2 = C[:, n-_hat_s+1:]

    # this is in numpy data type
    W = _construct_w(n, _hat_s)
    W, fake_W = cls_solving_mp(C_1, W)
    W_perp = C_2.transpose_conj()
    return W, fake_W, W_perp, C_1


def search_w(n, s):
    # params: n: number of workers
    # params: s: number of fail workers
    C = _construct_c(n)
    C = np.dot(1/np.sqrt(n), C)
    _hat_s = int(2*s+1)
    W = _construct_w(n, _hat_s)
    C_1 = C[:, 0:n-_hat_s+1]
    C_2 = C[:, n-_hat_s+1:]
    W, fake_W = _cls_solving(C_1, W)
    W_perp = _array_getH(C_2)
    # prepare matrix S
    s_tmp = np.zeros((1, n-_hat_s+1),dtype=complex)
    s_tmp[0][0] = 1.0+0.0j
    S = np.dot(s_tmp, _array_getH(C_1))
    return W, fake_W, W_perp, S, C_1, C


def _construct_c(n):
    # complex matrix here
    _shape = (n, n)
    C = np.zeros(_shape, dtype=complex)
    for p in range(_shape[0]):
        for q in range(_shape[1]):
            if q>=p:
                if p == 0 or q == 0:
                    C[p, q] = 1+0j
                else:
                    C[p, q]=0+np.exp(-2*np.pi*p*q*1j/n)
            else:
                C[p, q] = C[q, p]
    return C


def _construct_w(n, hat_s):
    _shape=(n, n)
    W = np.ones(_shape)
    for i in range(_shape[0]):
        if (i+hat_s) <= _shape[0]:
            _valid_range = range(i,i+hat_s)
        else:
            _valid_range = range(i,n)
            for t in range(i+hat_s-_shape[0]):
                _valid_range.append(t)
        for j in range(_shape[1]):
            if j not in _valid_range:
                W[i, j] = 0
    return W


def _cls_solving(C_1, fake_W):
    # return Q here:
    _shape = np.transpose(C_1).shape
    Q = np.ones(_shape,dtype=complex)
    for i in range(_shape[1]):
        indices = np.where(np.transpose(fake_W)[i]==0)[0]
        _A = np.zeros((len(indices),C_1.shape[1]-1),dtype=complex)
        _b = np.zeros((len(indices),1),dtype=complex)
        for j, index in enumerate(indices):
            _A[j] = C_1[index,1:]
            _b[j] = -C_1[index,0]
        _q=_cls_solver(_A, _b)
        Q[1:,i] = _q.reshape(Q[1:,i].shape)
    W = np.dot(C_1, Q)
    return W, fake_W


def _cls_solver(A, b):
    return np.dot(np.dot(np.linalg.inv(np.dot(_array_getH(A), A)), _array_getH(A)),b)


def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

In [89]:
from scipy import linalg as LA
from scipy import fftpack as FT

np.set_printoptions(precision=8,linewidth=200.0)
def obtain_E(alpha, E_2, n, s):
    # obtain E_1 in shape of n-2s by d
    _tmp_y = np.zeros((E_2.shape[1], n-s), dtype=complex)
    _tmp_y[:,0:s] = np.transpose(E_2)[:, -s:]
    for i in range(n-2*s):
        tmp = np.dot(_tmp_y[:,i:s+i], alpha)
        _tmp_y[:,s+i] = tmp.reshape(-1)
    return np.transpose(_tmp_y[:, s:])

def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

d=1
n=45
s=5

_poly_a = np.zeros(s+1, dtype=complex)
_poly_a[-1] = 1+0j

W, fake_W, W_perp, S, C_1, C = search_w(n=n, s=s)
simulated_G = np.random.randn(n, d)
simulated_adv = np.zeros((n, d),dtype=complex)

#simulated_adv[0:s,:] = -100+0j

simulated_R = np.dot(W, simulated_G) + simulated_adv
#random_vec = np.random.randn(d, 1)

#R_1 = np.dot(simulated_R, random_vec)

E_2 = np.dot(W_perp, simulated_R)
print(E_2)
print('======================================================')
print
_X_v3 = np.take(E_2, np.array([range(-i-(s+1), -i-2+1) for i in range(s)]))
tmp_y = np.take(E_2, np.array([-i-1 for i in range(s)]), axis=0)

alpha = LA.solve_toeplitz((_X_v3[:,0],_X_v3[0,:]), tmp_y)

_poly_a[0:s] = -alpha.reshape(-1)

recovered_E1 = obtain_E(alpha, E_2, n, s)
recovered_E = np.concatenate((recovered_E1, E_2), axis=0)

# let's generate true alpha
roots = [np.exp(2*np.pi*i*1j/n) for i in range(s)]
np_poly=P.polyfromroots(roots)


print("real poly a :")
print(np_poly)
print
print("our poly a:")
print(_poly_a)
print
print("recovered E:")
print(recovered_E)
print
print("correct E: ")
print(np.dot(_array_getH(C), simulated_adv))
print
print("recovered e (C * recovered_E): ")
print(np.dot(C, recovered_E))
print
print("recovered e (IFFT): ")
print(FT.ifft(recovered_E.reshape(-1)))
print
print
print(simulated_adv)

[[  4.56930138e-09 -1.14960130e-09j]
 [  4.71482053e-09 -1.49884727e-09j]
 [  3.08500603e-09 +5.96628524e-09j]
 [ -4.74392436e-09 -1.08557288e-08j]
 [ -1.29366526e-08 -2.27009878e-09j]
 [  1.06374500e-08 +4.65661287e-10j]
 [  1.20198820e-08 -3.95812094e-09j]
 [  1.27402018e-08 -7.39237294e-09j]
 [ -2.96768121e-09 +2.96859071e-09j]
 [  1.57160684e-09 -9.92440619e-09j]]

real poly a :
[-0.17364818-0.98480775j  2.14935983+4.40684071j -6.49771760-7.21644649j  8.23512927+5.14587989j -4.71312332-1.35146636j  1.00000000+0.j        ]

our poly a:
[-2.47554134-2.94658971j  0.24340563-0.10011464j  1.63027638-3.52216337j -1.94740010-0.31892322j  2.29492529+0.4108734j   1.00000000+0.j        ]

recovered E:
[[  1.32885785e-08 +1.18553054e-07j]
 [  5.78796723e-08 -2.82954179e-07j]
 [ -1.74879449e-07 +9.00621074e-07j]
 [  5.19621976e-07 -2.67286297e-06j]
 [ -1.99820984e-06 +8.23631012e-06j]
 [  6.64494911e-06 -2.47970283e-05j]
 [ -2.24595984e-05 +7.50002678e-05j]
 [  7.45098745e-05 -2.27111112e-04j]